In [9]:
import torch
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from deployment import preprocess, detect
import json
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

In [ ]:
device ="cuda" if torch.cuda.is_available() else "cpu" # use 'cuda:0' if GPU is available
# model_dir = "nealcly/detection-longformer" # model in our paper
model_dir = "yaful/DeepfakeTextDetection"  # model in the online demo
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)

In [4]:
def classify_text(text):
    # Detection
    result = detect(text, tokenizer, model, device)

    if (result == 'machine-generated'):
        return 1
    else:
        return 0

In [12]:
def predictor(ai_iclr, ai_neur, human_iclr, human_neur):
    """For prediction from the model"""
    ai_iclr = list(json.load(open(ai_iclr)).values())
    ai_neur = list(json.load(open(ai_neur)).values())
    human_iclr = list(json.load(open(human_iclr)).values())
    human_neur = list(json.load(open(human_neur)).values())

    ai_iclr_pred = [classify_text(text) for text in ai_iclr[608:]]
    ai_neur_pred = [classify_text(text) for text in ai_neur[576:]]
    human_iclr_pred = [classify_text(a) for text in human_iclr[608:] for a in text]
    human_neur_pred = [classify_text(a) for text in human_neur[576:] for a in text]

    actual = [1 for i in ai_iclr[608:]]  + [0 for revw in human_iclr[608:] for i in revw]
    predicted = ai_iclr_pred + human_iclr_pred

    print("++++++++++++++++ ICLR ++++++++++++++++")
    print("Confusion Matrix: ", confusion_matrix(actual, predicted))
    print("F1 Score: ", f1_score(actual, predicted))
    print("Precision: ", precision_score(actual, predicted))
    print("Recall: ", recall_score(actual, predicted))

    actual = [1 for i in ai_neur[576:]]  + [0 for revw in human_neur[576:] for i in revw]
    predicted = ai_neur_pred + human_neur_pred
    print("\n")
    print("++++++++++++++++ NEUR ++++++++++++++++")
    print("Confusion Matrix: ", confusion_matrix(actual, predicted))
    print("F1 Score: ", f1_score(actual, predicted))
    print("Precision: ", precision_score(actual, predicted))
    print("Recall: ", recall_score(actual, predicted))
    

## Actual Reviews:-

In [ ]:
predictor("../Dataset/ai_iclr.json","../Dataset/ai_neur.json","../Dataset/human_iclr.json","../Dataset/human_neur.json")

## Synonym Attack (Adjective Attack):-

In [ ]:
predictor("Dataset/Attacked Dataset/Adjective Attack/ai_iclr.json","Dataset/Attacked Dataset/Adjective Attack/ai_neur.json","Dataset/Attacked Dataset/Adjective Attack/human_iclr.json","Dataset/Attacked Dataset/Adjective Attack/human_neur.json")

## Paraphrasing Attack:-

In [ ]:
predictor("Dataset/Attacked Dataset/Paraphrasing Attack/para_ai_iclr.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_ai_neur.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_human_iclr.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_human_neur.json")

## Modified Paraphrasing Attack:-

In [ ]:
predictor("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/ai_iclr_para_modified.json","Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/ai_neur_para_modified.json","Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/human_iclr_para_modified.json","Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/human_neur_para_modified.json")